In [ ]:
%pip install hvac python-dotenv

In [ ]:
import hvac
import json
import os
import requests
from dotenv import load_dotenv

# Load the environment variables from the .env file
load_dotenv()

In [ ]:
client = hvac.Client(
    url=os.getenv('VAULT_API'),
    token=os.getenv('VAULT_TOKEN'),
)

# print(client.is_authenticated())

In [ ]:
try:
    secret_resp = client.secrets.kv.v2.read_secret_version(
        mount_point='kv', 
        path='homeassistant', 
        raise_on_deleted_version=False
    )
    
    if secret_resp['data'] is not None:
        secret_values = secret_resp['data']['data']
        ha_url = secret_values["API_URL"]
        ha_token = secret_values["ACCESS_TOKEN"]
    else:
        print("The secret does not exist.")
except hvac.exceptions.InvalidPath:
    print("The path is invalid or the permission is denied.")
except hvac.exceptions.Forbidden:
    print("The permission is denied.")
except hvac.exceptions.VaultError as e:
    print(f"Vault error occurred: {e}")

In [ ]:
def solar_production(input=""):
    # Define parameters for Home Assistant API call
    ha_entity_id = "sensor.solartoday"

    # Define headers for Home Assistant API call
    headers = {"Authorization": f"Bearer {ha_token}", "Content-Type": "application/json"}

    # Define endpoint and make API call to retrieve state of light entity
    url = f"{ha_url}/api/states/{ha_entity_id}"
    response = requests.get(url, headers=headers).content

    # Decode response and extract current state
    state = int(json.loads(response)["state"])
    
    if int(state) >= 1000:
        return(f"Today's Solar Production: {round(state/1000,2):,} kWh")
    else:
        return(f"Today's Solar Production: {int(state):,} Wh")

In [ ]:
print(solar_production())